In [1]:
from alab_management.builders import ExperimentBuilder
from alab_example import (
    Diffraction,
    Ending,
    Heating,
    PowderDosing,
    RecoverPowder,
)

from alab_control.labman import InputFile

/Users/yuxing/.pyenv/versions/3.10.11/envs/alab_management_new/lib/python3.10/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


Email alert is not set up due to either missing email_receivers, email_sender or email_password. It is also possible that the system is in simulation mode. Please recheck the config file if this is not expected.
Slack alert is not set up due to either missingslack_bot_token or slack_channel_id. It is also possible that the system is in simulation mode. Please recheck the config file if this is not expected.


In [2]:
exp = ExperimentBuilder(name="tutorial", tags=["tutorial"])

# Define the samples
samples = [
    exp.add_sample(f"tutorial_sample_{i}", tags=["tutorial"], precursors={"Li2CO3": 0.5 * i}) for i in range(1, 17)
]

# Define the steps
powder_dosing = PowderDosing(inputfiles={sample.name: InputFile(powder_dispenses=sample.metadata["precursors"]).to_json() for sample in samples})
powder_dosing.add_to(samples)

heating_1 = Heating(heating_time=120, heating_temperature=500)
heating_1.add_to(samples[:8])

heating_2 = Heating(heating_time=120, heating_temperature=600)
heating_2.add_to(samples[8:])

for sample in samples:
    powder_recovery = RecoverPowder()
    powder_recovery.add_to(sample)

    diffraction = Diffraction()
    diffraction.add_to(sample)

    ending = Ending()
    ending.add_to(sample)

In [31]:
import pymongo
import pandas as pd
from alab_management.utils.module_ops import load_definition
from alab_management.device_view.device import get_all_devices
import inspect
import re
load_definition()
all_devices = get_all_devices()

client = pymongo.MongoClient()
db = client["alab_example_sim"]
devices = db["devices"]
sample_positions = db["sample_positions"]

df = pd.DataFrame({}, index=["Device Type", "Communication Protocol", "Number of Devices", "Number of Associated Sample Positions", "Number of Exceptions"])

for device_type in devices.distinct("type"):
    n_devices = devices.count_documents({"type": device_type})
    n_positions = sum(sample_positions.count_documents({"name": {"$regex": rf"^{sp}"}}) for sp in devices.find_one({"type": device_type})["sample_positions"])
    device_obj = next(d for n, d in all_devices.items() if d.__class__.__name__ == device_type)
    source_code = inspect.getsource(device_obj.__class__)
    all_exceptions = re.findall(r"try:", source_code)
    n_exceptions = len(all_exceptions)

    df[device_type] = [device_type, "TCP/IP", n_devices, n_positions, n_exceptions]
df = df.T

In [32]:
df

,Device Type,Communication Protocol,Number of Devices,Number of Associated Sample Positions,Number of Exceptions
BallDispenser,BallDispenser,TCP/IP,1,1,1
BoxFurnace,BoxFurnace,TCP/IP,4,8,0
CapDispenser,CapDispenser,TCP/IP,1,3,0
CappingGripper,CappingGripper,TCP/IP,1,1,2
Diffractometer,Diffractometer,TCP/IP,1,1,2
LabmanQuadrant,LabmanQuadrant,TCP/IP,4,48,1
ManualFurnace,ManualFurnace,TCP/IP,4,8,0
RobotArmCharacterization,RobotArmCharacterization,TCP/IP,1,1,2
RobotArmFurnaces,RobotArmFurnaces,TCP/IP,1,1,2
Scale,Scale,TCP/IP,1,1,0


In [33]:
(df["Number of Devices"] * df["Number of Associated Sample Positions"]).sum()

289

In [39]:
from alab_management.task_view.task import get_all_tasks

all_tasks = get_all_tasks()


for task_name, task in all_tasks.items():
    source_code = inspect.getsource(task)
    all_exceptions = re.findall(r"try:", source_code)
    n_exceptions = len(all_exceptions)

    print(task_name, n_exceptions)

Starting 0
PowderDosing 7
Moving 1
Heating 3
ManualHeating 1
HeatingWithAtmosphere 1
RecoverPowder 17
PrepareSampleforXRD 9
Diffraction 3
Ending 6


In [44]:
table = r"""\begin{table}[]
\resizebox{\textwidth}{!}{%
\begin{tabular}{|c|c|c|c|}
\hline
\textbf{Task name} &
  \textbf{Capacity} &
  \textbf{\begin{tabular}[c]{@{}c@{}}Successfully\\ completed tasks\end{tabular}} &
  \textbf{\begin{tabular}[c]{@{}c@{}}Number of \\ Handled Errors\end{tabular}} \\ \hline
\textttPowderDosing          & 16 & 248  & 7  \\ \hline
\textttHeating               & 8  & 233  & 3  \\ \hline
\textttManualHeating         & 8  & 65   & 1  \\ \hline
\textttHeatingWithAtmosphere & 4  & 25   & 1  \\ \hline
\textttRecoverPowder         & 1  & 1865 & 17 \\ \hline
\textttDiffraction           & 1  & 1845 & 12 \\ \hline
\textttEnding                & 1  & 1752 & 6  \\ \hline
\end{tabular}%
}
\end{table}"""

print(re.sub(r"\\texttt(\w+)", r"\\texttt{\1}", table))

\begin{table}[]
\resizebox{\textwidth}{!}{%
\begin{tabular}{|c|c|c|c|}
\hline
\textbf{Task name} &
  \textbf{Capacity} &
  \textbf{\begin{tabular}[c]{@{}c@{}}Successfully\\ completed tasks\end{tabular}} &
  \textbf{\begin{tabular}[c]{@{}c@{}}Number of \\ Handled Errors\end{tabular}} \\ \hline
\texttt{PowderDosing}          & 16 & 248  & 7  \\ \hline
\texttt{Heating}               & 8  & 233  & 3  \\ \hline
\texttt{ManualHeating}         & 8  & 65   & 1  \\ \hline
\texttt{HeatingWithAtmosphere} & 4  & 25   & 1  \\ \hline
\texttt{RecoverPowder}         & 1  & 1865 & 17 \\ \hline
\texttt{Diffraction}           & 1  & 1845 & 12 \\ \hline
\texttt{Ending}                & 1  & 1752 & 6  \\ \hline
\end{tabular}%
}
\end{table}
